# Import package

In [1]:
import numpy as np
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load data

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train.shape
y_train.shape

print(x_train.shape, x_train.dtype," ", y_train.shape, y_train.dtype)

(50000, 32, 32, 3) uint8   (50000, 1) uint8


In [3]:
y_train = y_train.flatten()
y_test = y_test.flatten()

# Pretraining

In [4]:
# Normalization 
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Data Augment


In [5]:
import os
from PIL import Image

# dataflow
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.4,
    height_shift_range=0.4,
    horizontal_flip=True,
    zoom_range=0.4
)
data_generator = datagen.flow(x_train, y_train, batch_size=5000)

augmented_images, augmented_labels = [],[]
# ImageDataGenerator.flow() -> generate the rotate image
for i in range(5):  # run 5 batch
    x_batch, y_batch = next(data_generator)
    augmented_images.extend(x_batch)
    augmented_labels.extend(y_batch)


In [6]:
augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)

# Add the augment image to origin dataset
x_train_augment = np.concatenate((x_train, augmented_images), axis=0)
y_train_augment = np.concatenate((y_train, augmented_labels), axis=0)

# Training
VGG16: catch feature
SVM: classification

In [7]:
# Feature engineering
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
x = Flatten()(base_model.output)
feature_model = Model(inputs=base_model.input, outputs=x)

In [8]:
# check output
base_model.output.shape
x.shape
feature_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 32, 32, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 32, 32, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 16, 16, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 4, 4, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 2, 2, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 1, 1, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Predict
x_train_features = feature_model.predict(x_train_augment)
x_test_features = feature_model.predict(x_test)

2344/2344 ━━━━━━━━━━━━━━━━━━━━ 125s 53ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 18s 56ms/step


In [10]:
# Standardization
scaler = StandardScaler()
x_train_features = scaler.fit_transform(x_train_features)
x_test_features = scaler.fit_transform(x_test_features)

In [11]:
# check feature shape
x_train_features.shape, x_test_features.shape

((75000, 512), (10000, 512))

# SVM Classification

In [12]:
svm = SVC(kernel='rbf', C=10, gamma='scale')  # 使用 RBF 核函數
svm.fit(x_train_features, y_train_augment)

SVC(C=10)

In [13]:
# Estimate model
y_pred = svm.predict(x_test_features)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.606
Classification Report:
               precision    recall  f1-score   support

           0       0.65      0.73      0.69      1000
           1       0.64      0.68      0.66      1000
           2       0.57      0.49      0.52      1000
           3       0.43      0.45      0.44      1000
           4       0.58      0.58      0.58      1000
           5       0.52      0.53      0.52      1000
           6       0.62      0.65      0.63      1000
           7       0.69      0.65      0.67      1000
           8       0.73      0.71      0.72      1000
           9       0.64      0.60      0.62      1000

    accuracy                           0.61     10000
   macro avg       0.61      0.61      0.61     10000
weighted avg       0.61      0.61      0.61     10000



base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
x = Flatten()(base_model.output)
feature_model = Model(inputs=base_model.input, outputs=x)